In [1]:
import os
import csv
import time
import numpy as np
import pandas as pd
from math import sqrt
from itertools import product
from itertools import combinations_with_replacement

In [3]:
pip install openpyxl

     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     ---- -------------------------------- 30.7/250.0 kB 435.7 kB/s eta 0:00:01
     ---- -------------------------------- 30.7/250.0 kB 435.7 kB/s eta 0:00:01
     ---- -------------------------------- 30.7/250.0 kB 435.7 kB/s eta 0:00:01
     ---- -------------------------------- 30.7/250.0 kB 435.7 kB/s eta 0:00:01
     ------------- ----------------------- 92.2/250.0 kB 374.1 kB/s eta 0:00:01
     ---------------- ------------------- 112.6/250.0 kB 435.7 kB/s eta 0:00:01
     ---------------- ------------------- 112.6/250.0 kB 435.7 kB/s eta 0:00:01
     -------------------- --------------- 143.4/250.0 kB 369.8 kB/s eta 0:00:01
     ------------------------- ---------- 174.1/250.0 kB 402.6 kB/s eta 0:00:01
     ----------------------------- ------ 204.8/250.0 kB 429.5 kB/s eta 0:00:01
     --------------------------------- -- 235.5/250.0 kB 436.6 kB/s eta 0:00:01
     ------------------------------------ 250.0

In [4]:
# Load Data

df = pd.read_csv("Block_income_population_latlong_ward_commercialv3.csv")
df_school = pd.read_excel("a4_1 SchoolclipNblock.xlsx", engine='openpyxl')
df_commercial = pd.read_excel("CommercialZoneAreabyBuffer500mv2_LatLong.xlsx", engine='openpyxl')
df_commercial = df_commercial.fillna(0)

In [11]:
# Coordinates Data

I = list(df["BlockID_Flint"])
J = ['C2']
num_I = len(I)
I_J = J+I
G = list(df_commercial["C_ID"])

I_coordinates = {}
I_coordinates_x = list(df['Lat'])
I_coordinates_y = list(df['Long'])
for i in I:
    index = I.index(i)
    I_coordinates[i] = I_coordinates_x[index], I_coordinates_y[index]

J_coordinates = {}
for j in J:
    row = df_commercial[df_commercial['C_ID'] == j] 
    latitude = row['lat'].values[0] 
    longitude = row['long'].values[0] 
    J_coordinates[j] = latitude, longitude


coordinates = dict(I_coordinates, **J_coordinates)

coordinates

{'block1': (43.070266, -83.728011),
 'block10': (43.064229, -83.716173),
 'block100': (43.005303, -83.664936),
 'block101': (43.006735, -83.659094),
 'block102': (43.003117, -83.669835),
 'block103': (42.995855, -83.661379),
 'block104': (42.991694, -83.658892),
 'block105': (42.998919, -83.645679),
 'block106': (42.9919, -83.647173),
 'block107': (42.984229, -83.673224),
 'block108': (42.986067, -83.665428),
 'block109': (42.984999, -83.657968),
 'block11': (43.065026, -83.710028),
 'block110': (43.004942, -83.737069),
 'block111': (43.004941, -83.728728),
 'block112': (42.998789, -83.735249),
 'block113': (42.99943, -83.728243),
 'block114': (42.99647, -83.726548),
 'block115': (42.995573, -83.721052),
 'block116': (43.003622, -83.713312),
 'block117': (43.005907, -83.718585),
 'block118': (43.007208, -83.703648),
 'block119': (43.001267, -83.719109),
 'block12': (43.067583, -83.700383),
 'block120': (43.004249, -83.684511),
 'block121': (42.997711, -83.678069),
 'block122': (42.9960

In [6]:
# Distance Data

with open("CSV_routing_m.csv", newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    Routing_distances = {tuple(row[:2]): float(row[2]) for row in reader}
d = {}
d = Routing_distances

# # Population but no-need
# p = {}
# demand_elderly = df['E_M65_66'] + df['E_M67_69'] + df['E_M70_74'] + df['E_M75_79'] + df['E_M80_84'] + df['E_M85up'] 
# + df['E_F65_66'] + df['E_F67_69'] + df['E_F70_74'] + df['E_F75_79'] + df['E_F80_84'] + df['E_F85up']
# df['Elderly'] = demand_elderly
# for index, row in df.iterrows():
#     key = row["BlockID_Flint"]
#     value = row['Elderly']
#     p[key] = value
# cardinality_I = sum(p.values())


In [7]:
# OD matrix (distance)

OD_matrix = np.zeros((len(I_J), len(I_J)))

for i_index, i_id in enumerate(I_J):
    for j_index, j_id in enumerate(I_J):
        distance_key = (i_id, j_id)
        if distance_key in d:
            distance = d[distance_key]
        else:
            distance = np.inf
        OD_matrix[i_index, j_index] = distance


In [8]:
print(OD_matrix)
print(OD_matrix.shape)

[[    0.  8650.  6842. ...  6180.  4883.  2473.]
 [ 8640.     0.  1643. ... 13313. 12664. 12973.]
 [ 6833.  1643.     0. ... 11678. 11029. 10843.]
 ...
 [ 6763. 13398. 11708. ...     0.   751.  3397.]
 [ 4925. 12720. 11029. ...   751.     0.  3215.]
 [ 2343. 12993. 11381. ...  3341.  3147.     0.]]
(136, 136)


In [ ]:
# OD matrix CSV로 저장

# OD_df = pd.DataFrame(OD_matrix, index=I_J, columns=I_J)
# output_csv_path = "OD_matrix.csv"
# OD_df.to_csv(output_csv_path)